In [17]:
import subprocess
import sys
import boto3
# Retrieve the Jupyter Notebook token
jupyter_token = sys.argv[1] if len(sys.argv) > 1 else ""
# Retrieve the AWS region
session = boto3.session.Session()
current_region = session.region_name
# Retrieve the AWS account ID from the IAM user
sts_client = session.client('sts')
response = sts_client.get_caller_identity()
account_id = response['Account']
# Execute the Docker login command
docker_login_command = f"aws ecr get-login-password --region {current_region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{current_region}.amazonaws.com"
subprocess.run(["bash", "-c", docker_login_command], input=jupyter_token.encode("utf-8"))

Login Succeeded


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



CompletedProcess(args=['bash', '-c', 'aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 717913536206.dkr.ecr.us-east-1.amazonaws.com'], returncode=0)

In [ ]:
#Build the Docker container
docker_build_command = "docker build -t medical-mixtral-7b-v250k ."
subprocess.run(["bash", "-c", docker_build_command])



In [19]:

# Create a new ECR repository
ecr_repository_name = "aimedical"
create_ecr_repository_command = f"aws ecr create-repository --repository-name {ecr_repository_name} --region {current_region}"
subprocess.run(["bash", "-c", create_ecr_repository_command])


An error occurred (AccessDeniedException) when calling the CreateRepository operation: User: arn:aws:sts::717913536206:assumed-role/AmazonSageMakerServiceCatalogProductsUseRole/SageMaker is not authorized to perform: ecr:CreateRepository on resource: arn:aws:ecr:us-east-1:717913536206:repository/aimedical because no identity-based policy allows the ecr:CreateRepository action


CompletedProcess(args=['bash', '-c', 'aws ecr create-repository --repository-name aimedical --region us-east-1'], returncode=255)

In [20]:
# Tag the Docker image
tag_docker_image_command = f"docker tag medical-mixtral-7b-v250k:latest {account_id}.dkr.ecr.{current_region}.amazonaws.com/{ecr_repository_name}:latest"
subprocess.run(["bash", "-c", tag_docker_image_command])


CompletedProcess(args=['bash', '-c', 'docker tag medical-mixtral-7b-v250k:latest 717913536206.dkr.ecr.us-east-1.amazonaws.com/aimedical:latest'], returncode=0)

In [21]:
# Push the Docker image to ECR
push_docker_image_command = f"docker push {account_id}.dkr.ecr.{current_region}.amazonaws.com/{ecr_repository_name}:latest"
subprocess.run(["bash", "-c", push_docker_image_command])


The push refers to repository [717913536206.dkr.ecr.us-east-1.amazonaws.com/aimedical]
1c3b611570ca: Preparing
5416f99f3568: Preparing
15254b11f0a0: Preparing
fe680387eba2: Preparing
ce6e906ff40c: Preparing
71f226a3effa: Preparing
32f52ea0ee4b: Preparing
5aca128d25ca: Preparing
2814327c5674: Preparing
cb17d65a30d9: Preparing
9f2738d60323: Preparing
673fded1634e: Preparing
6c3e7df31590: Preparing
2814327c5674: Waiting
cb17d65a30d9: Waiting
9f2738d60323: Waiting
673fded1634e: Waiting
6c3e7df31590: Waiting
71f226a3effa: Waiting
32f52ea0ee4b: Waiting
5aca128d25ca: Waiting


denied: User: arn:aws:sts::717913536206:assumed-role/AmazonSageMakerServiceCatalogProductsUseRole/SageMaker is not authorized to perform: ecr:InitiateLayerUpload on resource: arn:aws:ecr:us-east-1:717913536206:repository/aimedical because no identity-based policy allows the ecr:InitiateLayerUpload action


CompletedProcess(args=['bash', '-c', 'docker push 717913536206.dkr.ecr.us-east-1.amazonaws.com/aimedical:latest'], returncode=1)

In [33]:

import boto3
from botocore.exceptions import ClientError

def bucket_exists(bucket_name):
    s3 = boto3.client('s3')
    try:
        response = s3.list_buckets()
        existing_buckets = [bucket['Name'] for bucket in response['Buckets']]
        return bucket_name in existing_buckets
    except ClientError as e:
        if e.response['Error']['Code'] == 'AccessDenied':
            print("Access denied when trying to list buckets.")
            return None
        else:
            raise

s3 = boto3.client('s3')
bucket_name = 'env-automatic'

bucket_status = bucket_exists(bucket_name)

if bucket_status is None:
    print("Skipping bucket creation due to access denial.")
elif not bucket_status:
    response = s3.create_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' created.")
    print(response)
else:
    print(f"Bucket '{bucket_name}' already exists.")


Access denied when trying to list buckets.
Skipping bucket creation due to access denial.


In [32]:
import sagemaker
# Get the default SageMaker execution role
role = sagemaker.get_execution_role()
print("SageMaker Execution Role ARN:", role)

SageMaker Execution Role ARN: arn:aws:iam::717913536206:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole


In [ ]:
import subprocess
import sys
import boto3
# Create a SageMaker Processing Job using the Docker Image
import boto3
sagemaker = boto3.client('sagemaker')
job_name = 'FineTuning'
image_uri = f"{account_id}.dkr.ecr.{current_region}.amazonaws.com/{ecr_repository_name}:latest"
output_s3_uri = f's3://{bucket_name}/output'

processing_job_request = {
    'ProcessingJobName': job_name,
    'RoleArn': role,
    'AppSpecification': {
        'ImageUri': image_uri,
    },
    'ProcessingOutputConfig': {
        'Outputs': [
            {
                'OutputName': 'output',
                'S3Output': {
                    'S3Uri': output_s3_uri,
                    'LocalPath': '/app/output',
                    'S3UploadMode': 'EndOfJob'
                }
            }
        ]
    },
    'ProcessingResources': {
        'ClusterConfig': {
            'InstanceCount': 1,
            'InstanceType': 'ml.m5.xlarge',
            'VolumeSizeInGB': 30
        }
    }
}
response = sagemaker.create_processing_job(**processing_job_request)
print(response)
